In [1]:
from pathlib import Path

Path('normalized').mkdir(exist_ok=True)

In [2]:
import pandas as pd
import numpy as np

# data_file_name = 'RetailGoodsAndPrices.csv'
data_file_name = 'RetailGoodsAndPricesSample.csv'

retail_goods_and_prices_csv = pd.read_csv(f'./data/{data_file_name}', sep='\t')

In [3]:
def process_data(df, category_name, output_file):
    
    # Get unique values for category and sort
    categories = df[category_name].unique()
    categories = [x for x in categories if str(x) != 'nan']
    categories = np.sort(categories)
    
    # Create dataframe
    columns = [category_name]
    cat_df = pd.DataFrame(categories, columns=columns)
    
    # Add ID column
    cat_df['ID'] = cat_df.index + 1
    
    # Output to CSV
    cat_df.to_csv(output_file, index=False, sep='\t', columns=['ID', category_name])

In [4]:
process_data(retail_goods_and_prices_csv, 'Group', './normalized/01_groups.csv')

In [5]:
groups = pd.read_csv('./normalized/01_groups.csv', sep='\t')
subgroups_columns = ['Group', 'Subgroup', 'Group_ID']
subgroups_df = pd.DataFrame(retail_goods_and_prices_csv, columns=subgroups_columns)
subgroups_df = subgroups_df.sort_values('Subgroup').drop_duplicates().reset_index()
    
# Add ID column
subgroups_df['ID'] = subgroups_df.index + 1

for index, row in subgroups_df.iterrows():
    group_id = groups[groups['Group'] == row['Group']].iloc[0]['ID']
    subgroups_df.loc[index, 'Group_ID'] = str(group_id)
    
# Output to CSV
subgroups_df.to_csv('./normalized/02_subgroups.csv', index=False, sep='\t', columns=['ID', 'Group_ID', 'Subgroup'])

In [6]:
process_data(retail_goods_and_prices_csv, 'Brand', './normalized/03_brands.csv')

In [7]:
subgroups = pd.read_csv('./normalized/02_subgroups.csv', sep='\t')
brands = pd.read_csv('./normalized/03_brands.csv', sep='\t')
items_columns = ['Code', 'Subgroup', 'Brand', 'Subgroup_ID', 'Brand_ID', 'Item', 'Description']
items_df = pd.DataFrame(retail_goods_and_prices_csv, columns=items_columns)


for index, row in items_df.iterrows():
    subgroup_id = subgroups[subgroups['Subgroup'] == row['Subgroup']].iloc[0]['ID']
    brand_id = brands[brands['Brand'] == row['Brand']].iloc[0]['ID']
    items_df.loc[index, 'Subgroup_ID'] = str(subgroup_id)
    items_df.loc[index, 'Brand_ID'] = str(brand_id)
    
# Output to CSV
items_df.to_csv('./normalized/04_items.csv', index=False, sep='\t', columns=['Code', 'Subgroup_ID', 'Brand_ID', 'Item', 'Description'])

In [8]:
retail_goods_and_prices_csv.to_csv('./normalized/05_wholesale_prices.csv', index=False, sep='\t', columns=['Code', 'Wholesale price (USD)'])